In [19]:
import boto3
from boto3.dynamodb.conditions import Key

In [74]:
# two options... can create boto3 client OR resource with DynamoDB

In [ ]:
ddb = boto3.client('dynamodb', endpoint_url='http://localhost:8000')
print(ddb.list_tables())

In [4]:
ddb = boto3.resource('dynamodb', endpoint_url='http://localhost:8000')
list(ddb.tables.all()) # prints empty list, as we have no tables

[dynamodb.Table(name='Employees')]

In [5]:
# we will use resources for this tutorial.
type(ddb)

boto3.resources.factory.dynamodb.ServiceResource

In [121]:
# create table using JSON format
table = ddb.create_table(
    TableName = 'Employees',
    # HASH and RANGE together make up a "primary key", which is unique in the table
    # you can have the same HASH multiple times, but with different RANGES (and vice versa?).
    KeySchema = [
        {
            'AttributeName': 'Name',
            'KeyType': 'HASH' # PARITION KEY
        },
        {
            'AttributeName': 'Email', # SORT KEY (optional)
            'KeyType': 'RANGE'
        }
    ],
    AttributeDefinitions = [
        {
            'AttributeName': 'Name',
            'AttributeType': 'S' # S == STRING
        },
        {
            'AttributeName': 'Email',
            'AttributeType': 'S' 
        }
    ],
    # doesn't matter for local instance
    ProvisionedThroughput={ # THE RATE AT WHICH YOU WANT YOUR READ/WRITE CAPACITY TO BE SET TO
        'ReadCapacityUnits': 1,
        'WriteCapacityUnits': 1
    },
    BillingMode='PAY_PER_REQUEST', # optional argument, does not matter for local instance
    # note that adding a single GSI doubles cost of writing,
    # and you can add a max of 20 GSIs. Replications to GSI table are quick
    # but not instantaneous.
    
    # also, write capacity on GSI table should honestly be HIGHER than 
    # write capacity on main table
    GlobalSecondaryIndexes=[
        {
            'IndexName': 'Country', # MUST BE UNIQUE ONLY FOR THIS TABLE
            # key schema must have at least a partition key. RANGE is optional
            'KeySchema': [
                {
                    'AttributeName': 'Email', # Seems like this needs to match the RANGE value from main table, just from what I've seen others do... need confirmation though
                    'KeyType': 'HASH'
                }
        ],
        # This represents attributes that are copied (projected) from main table into
        # secondary index table. These are in addition to PK atrributes and Index key attributes,
        # which are automatically projected.
            'Projection': {
                # can be 'KEYS_ONLY', 'ALL' or "INCLUDE", the latter of which requires a list to be passed elsewhere. See boto3 documentation.
                'ProjectionType': 'ALL'
                # 'NonKeyAttributes': ['string']
            },
            'ProvisionedThroughput': {
                'ReadCapacityUnits': 1,
                'WriteCapacityUnits': 2
            }
        }
    ]
)

print(table)

dynamodb.Table(name='Employees')


In [7]:
# ** if you are coming back to this tutorial and have already created the table...**
table = ddb.Table("Employees")

In [122]:
# returns key schema
table.key_schema

[{'AttributeName': 'Name', 'KeyType': 'HASH'},
 {'AttributeName': 'Email', 'KeyType': 'RANGE'}]

In [123]:
# returns the table attribute definitions
table.attribute_definitions

[{'AttributeName': 'Name', 'AttributeType': 'S'},
 {'AttributeName': 'Email', 'AttributeType': 'S'}]

In [124]:
# returned provisioned_throughput
table.provisioned_throughput

{'LastIncreaseDateTime': datetime.datetime(1969, 12, 31, 19, 0, tzinfo=tzlocal()),
 'LastDecreaseDateTime': datetime.datetime(1969, 12, 31, 19, 0, tzinfo=tzlocal()),
 'NumberOfDecreasesToday': 0,
 'ReadCapacityUnits': 0,
 'WriteCapacityUnits': 0}

In [125]:
# find GSIs
table.global_secondary_indexes

[{'IndexName': 'Country',
  'KeySchema': [{'AttributeName': 'Email', 'KeyType': 'HASH'}],
  'Projection': {'ProjectionType': 'ALL'},
  'IndexStatus': 'ACTIVE',
  'ProvisionedThroughput': {'ReadCapacityUnits': 0, 'WriteCapacityUnits': 0},
  'IndexSizeBytes': 0,
  'ItemCount': 0,
  'IndexArn': 'arn:aws:dynamodb:ddblocal:000000000000:table/Employees/index/Country'}]

In [126]:
# insert record
# ddb.put_item(TableName='Employees', Item={
#     'Name': {
#         'S':'Frey'
#     },
#     'Email': {
#         'S': 'jordan@freygeospatial.com'
#     },
#     'Country': {
#         'S', 'USA'
#     }
# })
table.put_item(Item={'Name':'Frey', 
               'Email': 'jordan@freygeospatial.com',
               'Country':'USA',
               'RandomAttribute': 'Blah!'})

{'ResponseMetadata': {'RequestId': 'ca22627d-72c1-4420-9fcd-ddd631d05203',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 06 May 2022 14:47:55 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2745614147',
   'x-amzn-requestid': 'ca22627d-72c1-4420-9fcd-ddd631d05203',
   'content-length': '2',
   'server': 'Jetty(9.4.18.v20190429)'},
  'RetryAttempts': 0}}

In [86]:
# scan table
results = table.scan()
for item in results:
    print(item)
    

Items
Count
ScannedCount
ResponseMetadata


In [107]:
results["Items"]

[{'Country': 'USA',
  'RandomAttribute': 'Blah!',
  'Email': 'jordan@freygeospatial.com',
  'Name': 'Frey'}]

In [88]:
results["Count"]

1

In [89]:
results["ScannedCount"]

1

In [101]:
results["ResponseMetadata"]

{'RequestId': '2a236310-bef8-4324-aa47-b0cc84e06265',
 'HTTPStatusCode': 200,
 'HTTPHeaders': {'date': 'Fri, 06 May 2022 13:43:05 GMT',
  'content-type': 'application/x-amz-json-1.0',
  'x-amz-crc32': '1839928699',
  'x-amzn-requestid': '2a236310-bef8-4324-aa47-b0cc84e06265',
  'content-length': '156',
  'server': 'Jetty(9.4.18.v20190429)'},
 'RetryAttempts': 0}

In [17]:
# we can request an item specifically if we have the Primary Key (both partition and sort keys). 
table.get_item(Key={'Name': 'Frey', 'Email': 'jordan@freygeospatial.com'}) # note that 'column names' are case sensitive!!

{'Item': {'Country': 'USA',
  'RandomAttribute': 'Blah!',
  'Email': 'jordan@freygeospatial.com',
  'Name': 'Frey'},
 'ResponseMetadata': {'RequestId': 'e3c11f42-9c63-4d90-819c-ec88b5d60d21',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 11 Aug 2022 20:35:29 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '4093131092',
   'x-amzn-requestid': 'e3c11f42-9c63-4d90-819c-ec88b5d60d21',
   'content-length': '126',
   'server': 'Jetty(9.4.18.v20190429)'},
  'RetryAttempts': 0}}

In [10]:
# add another record...
table.put_item(Item={'Name':'Frey', 
               'Email': 'Frey2@freygeospatial.com',
               'Country':'USA',
               'RandomAttribute': 'Ugh!'})

{'ResponseMetadata': {'RequestId': 'a579b84c-f986-46bc-9acf-d5c54abb7bbf',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 09 May 2022 19:29:02 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2745614147',
   'x-amzn-requestid': 'a579b84c-f986-46bc-9acf-d5c54abb7bbf',
   'content-length': '2',
   'server': 'Jetty(9.4.18.v20190429)'},
  'RetryAttempts': 0}}

In [13]:
# lets make sure we now have two records in the db
results = table.scan()
len(results["Items"])

2

In [15]:
# we can verify this further, if need be
results["Items"]

[{'Country': 'USA',
  'RandomAttribute': 'Ugh!',
  'Email': 'Frey2@freygeospatial.com',
  'Name': 'Frey'},
 {'Country': 'USA',
  'RandomAttribute': 'Blah!',
  'Email': 'jordan@freygeospatial.com',
  'Name': 'Frey'}]

In [8]:
# lets grab the first item, regardless of Partition Key
table.scan(Limit=1)

{'Items': [{'Country': 'USA',
   'RandomAttribute': 'Ugh!',
   'Email': 'Frey2@freygeospatial.com',
   'Name': 'Frey'}],
 'Count': 1,
 'ScannedCount': 1,
 'LastEvaluatedKey': {'Email': 'Frey2@freygeospatial.com', 'Name': 'Frey'},
 'ResponseMetadata': {'RequestId': 'fca64d7e-e405-4c8f-912a-9e586f1b16d9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 11 Aug 2022 20:14:48 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2783594237',
   'x-amzn-requestid': 'fca64d7e-e405-4c8f-912a-9e586f1b16d9',
   'content-length': '236',
   'server': 'Jetty(9.4.18.v20190429)'},
  'RetryAttempts': 0}}

In [118]:
# Remember that DynamoDB is SCHEMALESS -- we cannot specify
# columns other that the HASH (and maybe RANGE, which is optional when creating table).

# If we want to add a new item, we can specify any number of
# attributes there...

# to delete table D=
#table.delete()

# table.wait_until_not_exists() # check that table does not exist

In [13]:
# example of updating an item (table entry):
table.update_item(
    Key = {
        'Name': 'Frey',
        'Email': 'Frey2@freygeospatial.com'
    },
    UpdateExpression = 'SET Country = :updated',
    ExpressionAttributeValues = {':updated': 'Ukraine'}
)

{'ResponseMetadata': {'RequestId': 'f0ab37df-c5ee-4ca8-99f5-5b3a0e3d0e4e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 11 Aug 2022 20:28:30 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2745614147',
   'x-amzn-requestid': 'f0ab37df-c5ee-4ca8-99f5-5b3a0e3d0e4e',
   'content-length': '2',
   'server': 'Jetty(9.4.18.v20190429)'},
  'RetryAttempts': 0}}

In [16]:
# lets check to make sure the record was updated:
table.get_item(Key={'Name': 'Frey', 'Email': 'Frey2@freygeospatial.com'})['Item']

{'Country': 'Ukraine',
 'RandomAttribute': 'Ugh!',
 'Email': 'Frey2@freygeospatial.com',
 'Name': 'Frey'}

In [24]:
# example of a QUERY --  not just a 'get item'
response = table.query(
    KeyConditionExpression=Key('Name').eq('Frey')
)

items = response['Items']
for item in items:
    print(item)


{'Country': 'Ukraine', 'RandomAttribute': 'Ugh!', 'Email': 'Frey2@freygeospatial.com', 'Name': 'Frey'}
{'Country': 'USA', 'RandomAttribute': 'Blah!', 'Email': 'jordan@freygeospatial.com', 'Name': 'Frey'}


In [25]:
# example of a QUERY --  not just a 'get item'
response = table.query(
    KeyConditionExpression=Key('Name').eq('Frey') & Key('Email').eq('Frey2@freygeospatial.com') # Key val must actually be a deisgnated key (Hash and range). Cannot query on country
)

items = response['Items']
for item in items:
    print(item)


{'Country': 'Ukraine', 'RandomAttribute': 'Ugh!', 'Email': 'Frey2@freygeospatial.com', 'Name': 'Frey'}


In [26]:
# delete an item
response = table.delete_item(
    Key={
        'Name': 'Frey',
        'Email': 'Frey2@freygeospatial.com'
    }
)

In [27]:
# check that deletion was a success
table.scan()["Items"]

[{'Country': 'USA',
  'RandomAttribute': 'Blah!',
  'Email': 'jordan@freygeospatial.com',
  'Name': 'Frey'}]